In [ ]:
pip install torch numpy pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn

In [ ]:
from google.colab import files
uploaded =  files.upload()

Saving dataset.csv to dataset (1).csv


In [ ]:
# Load the dataset
data = pd.read_csv("dataset.csv")

In [ ]:
data.head()

,input,target
0,අපි පා ඩම් කරමි,අපි පා ඩම් කරමු
1,මම බත් කමු,අපි බත් කමු
2,අපි පන්සලට යති,අපි පන්සලට යමු
3,අපි ලියුමක් ලියයි,අපි ලියුමක් ලියමු
4,ඇය සින්දු කියමු,අපි සින්දු කියමු


In [ ]:
print(data.isnull().sum())

input     0
target    0
dtype: int64


In [ ]:
# Extract input and target sentences
input_texts = data["input"].tolist()
target_texts = data["target"].tolist()

In [ ]:
# Add start and end tokens to the target sentences
start_token = "<sos>"
end_token = "<eos>"
target_texts = [f"{start_token} {text} {end_token}" for text in target_texts]

In [ ]:
# Create vocabulary
def build_vocab(texts):
    vocab = Counter()
    for text in texts:
        vocab.update(text.split())
    word2idx = {word: idx for idx, (word, _) in enumerate(vocab.items(), start=1)}
    word2idx["<pad>"] = 0
    idx2word = {idx: word for word, idx in word2idx.items()}
    return word2idx, idx2word

In [ ]:
# Build vocab for input and target
input_word2idx, input_idx2word = build_vocab(input_texts)
target_word2idx, target_idx2word = build_vocab(target_texts)

In [ ]:
# Tokenize sentences
def tokenize_sentences(texts, word2idx):
    return [[word2idx[word] for word in text.split() if word in word2idx] for text in texts]


In [ ]:
input_sequences = tokenize_sentences(input_texts, input_word2idx)
target_sequences = tokenize_sentences(target_texts, target_word2idx)

In [ ]:
# Pad sequences
input_sequences = pad_sequence([torch.tensor(seq) for seq in input_sequences], batch_first=True, padding_value=0)
target_sequences = pad_sequence([torch.tensor(seq) for seq in target_sequences], batch_first=True, padding_value=0)


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, target_sequences, test_size=0.2, random_state=42)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, cell):
        embedded = self.embedding(x.unsqueeze(1))  # Add sequence dimension
        outputs, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        predictions = self.fc(outputs.squeeze(1))
        return predictions, hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.shape
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

In [ ]:
import torch.optim as optim

# Hyperparameters
embedding_dim = 256
hidden_dim = 512
input_vocab_size = len(input_word2idx)
target_vocab_size = len(target_word2idx)

In [ ]:
# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = Encoder(input_vocab_size, embedding_dim, hidden_dim).to(device)
decoder = Decoder(target_vocab_size, embedding_dim, hidden_dim).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
epochs = 35
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for i in range(len(X_train)):
        src = X_train[i].to(device)
        trg = y_train[i].to(device)

        optimizer.zero_grad()
        outputs = model(src.unsqueeze(0), trg.unsqueeze(0))

        outputs = outputs[1:].view(-1, outputs.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(outputs, trg)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(X_train):.4f}")

Epoch 1/35, Loss: 4.5288
Epoch 2/35, Loss: 3.5660
Epoch 3/35, Loss: 3.2538
Epoch 4/35, Loss: 2.8828
Epoch 5/35, Loss: 2.5875
Epoch 6/35, Loss: 2.4378
Epoch 7/35, Loss: 2.3440
Epoch 8/35, Loss: 2.2389
Epoch 9/35, Loss: 2.3768
Epoch 10/35, Loss: 2.3038
Epoch 11/35, Loss: 2.1514
Epoch 12/35, Loss: 2.0383
Epoch 13/35, Loss: 2.0351
Epoch 14/35, Loss: 1.9129
Epoch 15/35, Loss: 1.8264
Epoch 16/35, Loss: 1.6981
Epoch 17/35, Loss: 1.5676
Epoch 18/35, Loss: 1.4742
Epoch 19/35, Loss: 1.5738
Epoch 20/35, Loss: 1.4916
Epoch 21/35, Loss: 1.3541
Epoch 22/35, Loss: 1.3357
Epoch 23/35, Loss: 1.2074
Epoch 24/35, Loss: 1.1141
Epoch 25/35, Loss: 1.0383
Epoch 26/35, Loss: 0.9745
Epoch 27/35, Loss: 0.9199
Epoch 28/35, Loss: 0.7926
Epoch 29/35, Loss: 0.6827
Epoch 30/35, Loss: 0.5871
Epoch 31/35, Loss: 0.5131
Epoch 32/35, Loss: 0.4705
Epoch 33/35, Loss: 0.4198
Epoch 34/35, Loss: 0.3155
Epoch 35/35, Loss: 0.3123


In [ ]:
torch.save(model.state_dict(), 'model_nlp.pth')

In [146]:

def translate_sentence(sentence, model, input_word2idx, target_idx2word, max_len=50):
    model.eval()
    tokens = [input_word2idx[word] for word in sentence.split() if word in input_word2idx]
    src_tensor = torch.tensor(tokens).unsqueeze(0).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    outputs = [target_word2idx["<sos>"]]

    for _ in range(max_len):
        trg_tensor = torch.tensor([outputs[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        outputs.append(pred_token)
        if pred_token == target_word2idx["<eos>"]:
            break
    translated_sentence = " ".join([target_idx2word[idx] for idx in outputs[1:-1]])

    return translated_sentence

In [147]:
test_sentence_1 = "දකුණු පළාතේ නාකියාදෙණීය තෙල්ලඹුරේ ග්‍රාමයේ දී රත්න ශ්‍රී විජේසිංහ උපන්නෝය"
print(translate_sentence(test_sentence_1, model, input_word2idx, target_idx2word))

දකුණු පළාතේ නාකියාදෙණීය තෙල්ලඹුරේ ග්‍රාමයේ දී රත්න ශ්‍රී විජේසිංහ උපන්නේය


In [148]:
test_sentence_2 = "සීතල උඳුවප් මහේ සාමයේ කුමරුන්ගේ උපත පිළිබඳ පණිවුඩය රැගෙන මඳ නල දසත හමා යද්දී උදා වූ අසිරිමත් නත්තලෙහි ප්‍රීතියෙන් මුළු ලොව ම නැහැවී ගියහ"
print(translate_sentence(test_sentence_2, model, input_word2idx, target_idx2word))

සීතල උඳුවප් මහේ සාමයේ කුමරුන්ගේ උපත පිළිබඳ පණිවුඩය රැගෙන මඳ නල දසත හමා යද්දී උදා වූ අසිරිමත් නත්තලෙහි ප්‍රීතියෙන් මුළු ලොව ම නැහැවී ගියේය


In [149]:
test_sentence = "ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තෝය"
print(translate_sentence(test_sentence, model, input_word2idx, target_idx2word))

ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තේය


In [152]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
# Instead of accuracy_score, we will use nltk.translate.bleu_score
from nltk.translate.bleu_score import sentence_bleu

def translate_paragraph(paragraph, model, input_word2idx, target_idx2word, target_word2idx, max_len=50):
    # Tokenize the paragraph into sentences using sent_tokenize
    sentences = sent_tokenize(paragraph)

    # Translate each sentence
    translated_sentences = []
    for sentence in sentences:
        translated_sentence = translate_sentence(sentence, model, input_word2idx, target_idx2word, max_len)

        # Add a full stop if not already present
        if not translated_sentence.endswith('.'):
            translated_sentence += '.'

        translated_sentences.append(translated_sentence)

    # Join the translated sentences to form the paragraph
    translated_paragraph = " ".join(translated_sentences)

    # Calculate BLEU score instead of word-level accuracy
    original_tokens = [word for word in paragraph.split()]
    translated_tokens = [word for word in translated_paragraph.split()]
    # Use sentence_bleu for calculating the BLEU score
    bleu_score = sentence_bleu([original_tokens], translated_tokens)

    return translated_paragraph + f"\nAccuracy: {bleu_score:.2f}"


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [153]:
# Example input paragraph
paragraph = "දකුණු පළාතේ නාකියාදෙණීය තෙල්ලඹුරේ ග්‍රාමයේ දී රත්න ශ්‍රී විජේසිංහ උපන්නෝය. ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කරමි. ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තෝය."

# Translate the paragraph
translated_paragraph = translate_paragraph(paragraph, model, input_word2idx, target_idx2word, target_word2idx)

print("Translated Paragraph:")
print(translated_paragraph)


Translated Paragraph:
දකුණු පළාතේ නාකියාදෙණීය තෙල්ලඹුරේ ග්‍රාමයේ දී රත්න ශ්‍රී විජේසිංහ උපන්නේය. ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කළේය. ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තේය.
Accuracy: 0.86


In [ ]:
pip install gradio

In [154]:
import gradio as gr

# Function to integrate with Gradio
def translate_with_gradio(input_text):
    # Check if the input is a single sentence or a paragraph
    translated_text = translate_paragraph(input_text, model, input_word2idx, target_idx2word, target_word2idx)
    return translated_text

# Create Gradio interface
interface = gr.Interface(
    fn=translate_with_gradio,
    inputs=gr.Textbox(lines=10, placeholder="Enter a Sinhala sentence or paragraph for corrected..."),
    outputs=gr.Textbox(lines=10, label="Corrected Text"),
    title="Sinhala Grammar corrector",
    description="This is a Seq2Seq-based translation model. Enter Sinhala text to get the corrected output."
)

# Launch the Gradio app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3bf5f05c9c8dd812f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
